In [4]:
! pip install pytesseract pillow pandas pdf2image


In [2]:
!apt-get update
!apt-get install -y poppler-utils

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]          
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,937 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,233 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [48.5 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,174 kB]
Get:12 https://ppa.launchpadcontent.net/

In [ ]:
pip install pymupdf pdf2image pytesseract pillow


In [ ]:
import pytesseract
from pdf2image import convert_from_path

# Input / Output
pdf_path = "/kaggle/input/scanned-data-set/Revoult Bank statement.pdf"   # <-- your PDF
output_html = "bank_statement_full.html"

# Convert PDF → Images
pages = convert_from_path(pdf_path, dpi=200)
html_parts = []

for page_num, page in enumerate(pages, start=1):
    # OCR with line-level data
    data = pytesseract.image_to_data(page, output_type=pytesseract.Output.DICT)
    page_w, page_h = page.size

    # Page container
    html_parts.append(
        f"<div class='page' style='position:relative;width:{page_w}px;height:{page_h}px;"
        "border:1px solid #ccc;margin:20px auto;background:white;'>"
    )

    current_line = None
    line_text, line_left, line_top = [], 0, 0

    for i, word in enumerate(data["text"]):
        word = word.strip()
        if not word:
            continue

        line_no = (data["block_num"][i], data["par_num"][i], data["line_num"][i])
        x, y = data["left"][i], data["top"][i]

        if line_no != current_line:
            # flush previous line
            if line_text:
                html_parts.append(
                    f"<div style='position:absolute;left:{line_left}px;top:{line_top}px;"
                    "font-size:12px;white-space:nowrap;'>"
                    f"{' '.join(line_text)}</div>"
                )
            # start new line
            current_line = line_no
            line_text, line_left, line_top = [word], x, y
        else:
            line_text.append(word)

    # flush last line
    if line_text:
        html_parts.append(
            f"<div style='position:absolute;left:{line_left}px;top:{line_top}px;"
            "font-size:12px;white-space:nowrap;'>"
            f"{' '.join(line_text)}</div>"
        )

    html_parts.append("</div>")  # end page

# Final HTML skeleton
html_output = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Bank Statement</title>
<style>
  body {{ background:#f9f9f9; font-family:Arial, sans-serif; }}
  .page {{ box-shadow:0 0 8px rgba(0,0,0,0.2); }}
</style>
</head>
<body>
{''.join(html_parts)}
</body>
</html>
"""

with open(output_html, "w", encoding="utf-8") as f:
    f.write(html_output)

print(f"✅ Full-page HTML saved to {output_html}")


# another try

In [ ]:
import pytesseract
from pdf2image import convert_from_path

pdf_path = "/kaggle/input/scanned-data-set/Business Bank statment.pdf"
output_html = "bank_statement_page.html"

# Convert PDF to images
pages = convert_from_path(pdf_path, dpi=200)
html_parts = []

for page_num, page in enumerate(pages, start=1):
    data = pytesseract.image_to_data(page, output_type=pytesseract.Output.DICT)
    page_w, page_h = page.size

    # Page container
    html_parts.append(
        f"<div class='page' style='position:relative;width:{page_w}px;height:{page_h}px;"
        "border:1px solid #ccc;margin:20px auto;background:white;'>"
    )

    current_line, line_text, line_x, line_y = None, [], 0, 0

    for i, word in enumerate(data["text"]):
        word = word.strip()
        if not word:
            continue
        line_no = (data["block_num"][i], data["par_num"][i], data["line_num"][i])
        x, y = data["left"][i], data["top"][i]

        if line_no != current_line:
            if line_text:  # flush previous line
                html_parts.append(
                    f"<div style='position:absolute;left:{line_x}px;top:{line_y}px;"
                    "font-size:12px;white-space:nowrap;'>"
                    f"{' '.join(line_text)}</div>"
                )
            current_line, line_text, line_x, line_y = line_no, [word], x, y
        else:
            line_text.append(word)

    if line_text:  # flush last line
        html_parts.append(
            f"<div style='position:absolute;left:{line_x}px;top:{line_y}px;"
            "font-size:12px;white-space:nowrap;'>"
            f"{' '.join(line_text)}</div>"
        )

    html_parts.append("</div>")  # end page

# Wrap in HTML skeleton
html_output = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Bank Statement</title>
<style>
  body {{ background:#f9f9f9; font-family:Arial, sans-serif; }}
  .page {{ box-shadow:0 0 8px rgba(0,0,0,0.2); }}
</style>
</head>
<body>
{''.join(html_parts)}
</body>
</html>
"""

with open(output_html, "w", encoding="utf-8") as f:
    f.write(html_output)

print(f" Full page-like HTML saved to {output_html}")


In [ ]:
# === requirements (kaggle/linux) ===
# !apt-get install -y poppler-utils tesseract-ocr
# !pip install pdf2image pytesseract pillow

import html
import re
import pytesseract
from pdf2image import convert_from_path

# (kaggle fix) uncomment if needed
# pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

PDF_PATH   = "/kaggle/input/scanned-data-set/Business Bank statment.pdf"   # <- your input PDF
OUTPUT_HTML = "bank_statement_paragraphs.html"

# convert each pdf page to an image (higher dpi -> better OCR)
pages = convert_from_path(PDF_PATH, dpi=300)

# date pattern to split merged header + first row (handles: 07 Jun, 07 Jun 2019, 07/06/2019, 2019-06-07)
DATE_PAT = r'(\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b|\b\d{4}[/-]\d{1,2}[/-]\d{1,2}\b|\b\d{1,2}\s+[A-Za-z]{3}(?:\s+\d{2,4})?\b)'

html_pages = []

for page_idx, page_img in enumerate(pages, start=1):
    # get plain text (already grouped by lines reasonably well)
    raw_text = pytesseract.image_to_string(page_img)
    lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]

    # fix cases where OCR merges header + first transaction into one line:
    fixed_lines = []
    for ln in lines:
        # split where a new date-like token starts, but keep the split token (lookahead)
        parts = re.split(f"(?={DATE_PAT})", ln)
        for p in parts:
            p = p.strip()
            if p:
                fixed_lines.append(p)

    # build <p> paragraphs for this page
    para_html = "\n".join(f"<p>{html.escape(ln)}</p>" for ln in fixed_lines)

    html_pages.append(
        f"""
        <section class="page">
            <h2>Page {page_idx}</h2>
            {para_html if para_html.strip() else "<p><em>(No text detected)</em></p>"}
        </section>
        """
    )

# wrap into a simple, clean HTML file
final_html = f"""<!doctype html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>Bank Statement (OCR)</title>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
  body {{ font-family: Arial, sans-serif; background:#f7f7f7; margin:0; }}
  .container {{ max-width: 900px; margin: 24px auto; background:#fff; padding: 24px; box-shadow: 0 4px 18px rgba(0,0,0,0.08); border-radius: 12px; }}
  .page {{ margin: 24px 0; }}
  .page h2 {{ font-size: 16px; color:#444; margin: 0 0 12px; border-bottom:1px solid #eee; padding-bottom:8px; }}
  p {{ margin: 8px 0; line-height: 1.45; word-break: break-word; }}
</style>
</head>
<body>
  <div class="container">
    <h1>Bank Statement (Scanned → HTML)</h1>
    {"".join(html_pages)}
  </div>
</body>
</html>"""

with open(OUTPUT_HTML, "w", encoding="utf-8") as f:
    f.write(final_html)

print(f"✅ HTML created: {OUTPUT_HTML}")


# try with another format 

In [ ]:
pip install pymupdf


In [ ]:
import fitz  # PyMuPDF for PDF reading
import pytesseract
from PIL import Image
import io, re, json

# If tesseract is not in PATH, set the path manually:
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

pdf_path = "/kaggle/input/scanned-data-set/Revoult Bank statement.pdf"

def pdf_to_images(pdf_path):
    """Convert each PDF page to an image"""
    pdf = fitz.open(pdf_path)
    images = []
    for page_num in range(len(pdf)):
        page = pdf[page_num]
        pix = page.get_pixmap(dpi=300)
        img = Image.open(io.BytesIO(pix.tobytes("png")))
        images.append(img)
    return images

def extract_text_from_images(images):
    """OCR each page image"""
    texts = []
    for img in images:
        text = pytesseract.image_to_string(img)
        texts.append(text)
    return "\n".join(texts)

def parse_transactions(text):
    """Parse OCR text into structured transactions"""
    transactions = []
    lines = text.split("\n")
    
    # Example regex for dates like 01/06/2020 or 01 Jun 2020
    date_pattern = re.compile(r"(\d{1,2}[-/\s]?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|\d{1,2})[-/\s]?\d{2,4})")
    
    current = {}
    for line in lines:
        if date_pattern.search(line):
            if current:  # save previous transaction
                transactions.append(current)
                current = {}
            current["Date"] = date_pattern.search(line).group(1)
            
            # Remove date from line, keep description/amount
            desc_line = line.replace(current["Date"], "").strip()
            current["Description"] = desc_line
        else:
            # Continue description if line doesn't start with a date
            if "Description" in current:
                current["Description"] += " " + line.strip()
        
        # Look for money values
        money_match = re.findall(r"[\d,]+\.\d{2}", line)
        if money_match:
            # Simple heuristic: last value = balance
            if len(money_match) == 3:
                current["Money Out"] = money_match[0]
                current["Money In"] = money_match[1]
                current["Balance"] = money_match[2]
            elif len(money_match) == 2:
                current["Money Out"] = money_match[0]
                current["Balance"] = money_match[1]
    
    if current:
        transactions.append(current)

    return transactions

# ---------------- Run the pipeline ----------------
images = pdf_to_images(pdf_path)
text = extract_text_from_images(images)
transactions = parse_transactions(text)

# Save as JSON
with open("transactions.json", "w") as f:
    json.dump(transactions, f, indent=4)

print(" Extracted transactions saved to transactions.json")


In [ ]:
import re, json

# Load raw OCR JSON
with open("/kaggle/working/transactions.json", "r") as f:
    data = json.load(f)

cleaned = []
date_pattern = re.compile(r"^\d{1,2}\s?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s?\d{2,4}$")

skip_keywords = ["Grand Total", "Opening Balance", "Closing Balance", 
                 "Sort Code", "Customer Id", "Currency", "Account Number"]

for row in data:
    date = row.get("Date", "").strip()
    desc = row.get("Description", "").strip()

    # 1. Keep only rows with valid dates
    if not date_pattern.match(date):
        continue

    # 2. Skip metadata rows
    if any(keyword in desc for keyword in skip_keywords):
        continue

    # 3. Extract money values
    money_matches = re.findall(r"\d{1,3}(?:,\d{3})*\.\d{2}", desc)
    money_in, money_out, balance = None, None, None

    if len(money_matches) == 2:  # Out + Balance
        money_out, balance = money_matches
    elif len(money_matches) == 3:  # Out, In, Balance
        money_out, money_in, balance = money_matches
    elif len(money_matches) == 1:  # Just Balance (rare)
        balance = money_matches[0]

    cleaned.append({
        "Date": date,
        "Description": desc,
        "Money Out": money_out,
        "Money In": money_in,
        "Balance": balance
    })

# Save cleaned JSON
with open("transactions_cleaned.json", "w") as f:
    json.dump(cleaned, f, indent=4)

print("✅ Cleaned transactions saved to transactions_cleaned.json")


# csv output

In [6]:
# Install needed packages if not installed:
# !pip install pdf2image pytesseract pandas pillow opencv-python openpyxl

from pdf2image import convert_from_path
from PIL import Image
import pytesseract
import pandas as pd
import re
import os
import cv2
import numpy as np

pdf_path = '/kaggle/input/scanned-data-set/Business Bank statment.pdf'  # Change this to your bank statement file path
img_folder = 'pdf_pages'
os.makedirs(img_folder, exist_ok=True)

# 1. Convert PDF pages to images
pages = convert_from_path(pdf_path, dpi=300)
image_paths = []
for idx, page in enumerate(pages):
    img_file = os.path.join(img_folder, f'page_{idx}.png')
    page.save(img_file, 'PNG')
    image_paths.append(img_file)

# 2. OCR with image preprocessing and accumulate text
full_text = ""
for img_file in image_paths:
    # Load in grayscale
    img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    
    # Apply binarization for better OCR
    _, img_bin = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Convert back to PIL Image for pytesseract
    processed_img = Image.fromarray(img_bin)

    # OCR extraction
    text = pytesseract.image_to_string(processed_img, lang='eng')
    full_text += text + "\n"

# 3. Save raw OCR text for debugging
with open('raw_text.txt', 'w', encoding='utf-8') as f:
    f.write(full_text)
print("Raw OCR text saved to raw_text.txt. Please check this file to adjust regex.")

# 4. Define regex pattern to extract transactions
# Example pattern for date, description, amount, balance:
# Adjust this pattern based on your raw_text.txt inspection

pattern = r"(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})\s+(.+?)\s+([-+]?\$?[\d,]+\.\d{2})\s+([\$?[\d,]+\.\d{2})"
# If the above pattern does not match your format, update it accordingly.

transactions = re.findall(pattern, full_text)

data = []
for t in transactions:
    date, desc, amt, bal = t
    # Clean extracted values
    amt = amt.replace(',', '').replace('$', '')
    bal = bal.replace(',', '').replace('$', '')
    data.append({'Date': date.strip(), 'Description': desc.strip(), 'Amount': amt.strip(), 'Balance': bal.strip()})

# 5. Check if any transactions found
if not data:
    print("No transactions extracted. Please check raw_text.txt and adjust the regex pattern accordingly.")
else:
    # 6. Save extracted data as CSV and Excel
    df = pd.DataFrame(data)
    df.to_csv('statement.csv', index=False)
    df.to_excel('statement.xlsx', index=False)
    print("Extraction successful!")
    print("CSV saved as statement.csv")
    print("Excel saved as statement.xlsx")


Raw OCR text saved to raw_text.txt. Please check this file to adjust regex.
No transactions extracted. Please check raw_text.txt and adjust the regex pattern accordingly.


In [7]:
import re
import pandas as pd

# Sample raw_text you got loaded into a variable
with open('/kaggle/working/raw_text.txt', 'r', encoding='utf-8') as f:
    full_text = f.read()

# Regex pattern that:
# - captures date as day + month (with optional space)
# - captures description possibly multi-word until amounts start
# - groups optional "Money out", "Money in", and "Balance" columns

pattern = re.compile(
    r'(\d{1,2}\s?[A-Za-z]{3})\s+'                   # Date, e.g. "7Jun" or "10 Jun"
    r'(.+?)\s+'                                     # Description (non-greedy to stop before amounts)
    r'(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)?\s*'        # Optional Money out (numbers with commas and decimals)
    r'(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)?\s+'        # Optional Money in
    r'(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)'             # Balance (always present)
)

matches = pattern.findall(full_text)

data = []
for m in matches:
    date, desc, money_out, money_in, balance = m
    # Clean and convert amounts, empty strings to None
    money_out = money_out.replace(',', '') if money_out else ''
    money_in = money_in.replace(',', '') if money_in else ''
    balance = balance.replace(',', '') if balance else ''
    data.append({
        'Date': date.strip(),
        'Description': desc.strip(),
        'Money Out (£)': money_out.strip(),
        'Money In (£)': money_in.strip(),
        'Balance (£)': balance.strip()
    })

if not data:
    print("No transactions found with current regex.")
else:
    df = pd.DataFrame(data)
    df.to_csv('statement_extracted.csv', index=False)
    df.to_excel('statement_extracted.xlsx', index=False)
    print(f"Extracted {len(data)} transactions.")
    print("Saved to statement_extracted.csv and statement_extracted.xlsx")


Extracted 3 transactions.
Saved to statement_extracted.csv and statement_extracted.xlsx
